# Reimplementing CanICA without nilearn.decompistion.canICA

In [1]:
%matplotlib inline

In [2]:
from nilearn import datasets
#dir='/raid/nilearn_data'
adhd_dataset = datasets.fetch_adhd(n_subjects=3,data_dir='/home/share/TmpData/Qinglin/nilearn_data/')
func_filenames = adhd_dataset.func  # list of 4D nifti files for each subject

# print basic information on the dataset
print('First functional nifti image (4D) is at: %s' %
      func_filenames[0])  # 4D data

First functional nifti image (4D) is at: /home/share/TmpData/Qinglin/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz


/usr/local/lib/python3.5/dist-packages/nilearn/datasets/func.py:503: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  dtype=None)


In [3]:
from nilearn._utils.niimg_conversions import _resolve_globbing
imgs = _resolve_globbing(func_filenames)

In [4]:
from nilearn.masking import compute_epi_mask
mask_img = compute_epi_mask(func_filenames)

from nilearn.input_data import NiftiMasker
masker = NiftiMasker(mask_img=mask_img, 
                     standardize=True,
                     detrend=1,
                     smoothing_fwhm=6., 
                     memory_level=2)
fmri_masked = masker.fit()

/home/uga_qinglin/.local/lib/python3.5/site-packages/nibabel/nifti1.py:582: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  ext_def = np.fromstring(ext_def, dtype=np.int32)
/usr/local/lib/python3.5/dist-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None
/usr/local/lib/python3.5/dist-packages/nilearn/_utils/cache_mixin.py:309: UserWarning: Caching has been enabled (memory_level = 2) but no Memory object or path has been provided (parameter memory). Caching deactivated for function resample_img.
  **kwargs)


In [12]:
masker

NiftiMasker(detrend=1, dtype=None, high_pass=None, low_pass=None,
      mask_args=None,
      mask_img=<nibabel.nifti1.Nifti1Image object at 0x7f2320216f98>,
      mask_strategy='background', memory=Memory(location=None),
      memory_level=2, sample_mask=None, sessions=None, smoothing_fwhm=6.0,
      standardize=True, t_r=None, target_affine=None, target_shape=None,
      verbose=0)

In [10]:
from nilearn.decomposition.base import mask_and_reduce

data = mask_and_reduce(
    masker, imgs, confounds=None,
    reduction_ratio=0.1,
    random_state=0,
    memory="/storage/nilearn_cache",
    memory_level=max(0, 2 + 1),
    n_jobs=1)



AttributeError: '_ExtractionFunctor' object has no attribute '__name__'

In [ ]:
from nilearn.decomposition import CanICA
canica = CanICA(mask=masker, n_components=20, smoothing_fwhm=6.,
                memory_level=2,
                threshold=3., verbose=10, random_state=0)
canica._raw_fit(data)
components = canica.components_

S = (components ** 2).sum(axis=1)
S[S == 0] = 1
components /= S[:, np.newaxis]

In [ ]:
from sklearn.decomposition import dict_learning_online
dictionary, components=dict_learning_online(data,n_components=30, 
                                            dict_init=components,
                                            alpha=10,shuffle=1,
                                            batch_size=20,method="cd",
                                            verbose=1)


In [ ]:
#dont run
save=components

In [ ]:
components=save

In [ ]:

from scipy import stats
components=stats.zscore(components,axis=1)



In [ ]:
# Unit-variance scaling
S = np.sqrt(np.sum(components ** 2, axis=1))
S[S == 0] = 1
components /= S[:, np.newaxis]

In [ ]:

import numpy as np
from scipy.stats import scoreatpercentile

abs_components = np.abs(components)
ratio=100. - (100. / len(components)) *3
threshold = scoreatpercentile(abs_components,ratio)
components[abs_components < threshold] = 0

for component in components:

    if np.sum(component > 0) < np.sum(component < 0):
        component *= -1

In [ ]:
#components_img=[masker.inverse_transform(component) for component in components]
components_img=masker.inverse_transform(components)

from nilearn.image import iter_img
from nilearn.plotting import plot_stat_map, show

for i, cur_img in enumerate(iter_img(components_img)):
    plot_stat_map(cur_img, display_mode="ortho", title="IC %d" % i,
                  cut_coords=None, colorbar='bwr')

show()

In [ ]:
# Import Region Extractor algorithm from regions module
# threshold=0.5 indicates that we keep nominal of amount nonzero voxels across all
# maps, less the threshold means that more intense non-voxels will be survived.
from nilearn.regions import RegionExtractor

extractor = RegionExtractor(components_img, threshold=0.5,
                            thresholding_strategy='ratio_n_voxels',
                            extractor='local_regions',
                            standardize=True, min_region_size=2000)
# Just call fit() to process for regions extraction
extractor.fit()
# Extracted regions are stored in regions_img_
regions_extracted_img = extractor.regions_img_
# Each region index is stored in index_
regions_index = extractor.index_
# Total number of regions extracted
n_regions_extracted = regions_extracted_img.shape[-1]

from nilearn.plotting import plot_prob_atlas
# Visualization of region extraction results
title = ('%d regions are extracted from %s components.'
         '\nEach separate color of region indicates extracted region'
         % (n_regions_extracted, 'all'))
plot_prob_atlas(regions_extracted_img, view_type='filled_contours',
                         title=title)